# Import required packages

In [1]:
#load packages
import pandas as pd
from functools import partial
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [2]:
import json
import tiktoken
import nltk

In [3]:
import chromadb
from sentence_transformers import SentenceTransformer

In [4]:
from transformers import AutoProcessor, AutoTokenizer, AutoModelForImageTextToText
import torch

In [5]:
from huggingface_hub import login
from dotenv import load_dotenv

In [6]:
import torchvision

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [46]:

import torch_neuronx
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer


ModuleNotFoundError: No module named 'torch_neuronx'

# Load in Documents

In this section we will load in the scraped documents and clean them.

In [8]:
#read in csv for majors
majors = pd.read_csv("uva_majors.csv")

#remove non major data
majors = majors.iloc[8:-1]


In [9]:
# remove the extra text from the text column
test = 'Info For Students Alumni Military Affiliated Students Faculty & Staff Search Search Submit Search Close search Info For Students Alumni Military Affiliated Students Faculty & Staff Calendars Academic Calendars Exam Schedules Student Records Diplomas Transcripts FERPA Programs Calendar Schools University Registrar Carruthers Hall, 1001 N. Emmet St. P.O. Box 400203 Charlottesville, VA 22904-4203 Staff Directory Contact Info Phone: (434) 924-4122 Fax: (434) 924-4156 Email: [email protected] Hours Of Operation M-F: 10am - noon and 1pm - 4pm Â© 2024 By the Rector and Visitors of the University of Virginia Legal Privacy Report a Barrier Share Your Feedback University of Virginia Mar 31, 2025 Undergraduate Record 2024-2025 Select a Catalog Undergraduate Record 2024-2025 Graduate Record 2024-2025 Global Search Catalog Search Choose Search Location Select an option Courses Programs Schools/Colleges & Departments Policies and Other Non-Academic Content Entire Catalog Search Keyword Field Whole Word/Phrase Advanced Search Catalog Navigation Catalog Home Academic Calendar Admission Schools Programs, Degrees & Course Info Student Resources ROTC University Regulations About UVA Archived Records HELP Undergraduate Record 2024-2025'
text_to_remove_head = 'Info For Students Alumni Military Affiliated Students Faculty & Staff Search Search Submit Search Close search Info For Students Alumni Military Affiliated Students Faculty & Staff Calendars Academic Calendars Exam Schedules Student Records Diplomas Transcripts FERPA Programs Calendar Schools University Registrar Carruthers Hall, 1001 N. Emmet St. P.O. Box 400203 Charlottesville, VA 22904-4203 Staff Directory Contact Info Phone: (434) 924-4122 Fax: (434) 924-4156 Email: [email protected] Hours Of Operation M-F: 10am - noon and 1pm - 4pm Â© 2024 By the Rector and Visitors of the University of Virginia Legal Privacy Report a Barrier Share Your Feedback University of Virginia Mar 12, 2025 Undergraduate Record 2024-2025 Select a Catalog Undergraduate Record 2024-2025 Graduate Record 2024-2025 Global Search Catalog Search Choose Search Location Select an option Courses Programs Schools/Colleges & Departments Policies and Other Non-Academic Content Entire Catalog Search Keyword Field Whole Word/Phrase Advanced Search Catalog Navigation Catalog Home Academic Calendar Admission Schools Programs, Degrees & Course Info Student Resources ROTC University Regulations About UVA Archived Records HELP Undergraduate Record 2024-2025' 
text_to_remove_tail = 'Back to Top | Print-Friendly Page (opens a new window) All catalogs © 2025 University of Virginia. Powered by Modern Campus Catalog™ . .'
#function to remove the text
def remove_text(text):
    if text.startswith(text_to_remove_head):
        text = text[len(text_to_remove_head):]
    if text.startswith(test):
        text = text[len(test):]
    if text.endswith(text_to_remove_tail):
        text = text[:-len(text_to_remove_tail)]
    return text.strip()

#apply the function to the text column
majors['Text'] = majors['Text'].apply(remove_text)

#since some of the majors have extra text that is a bit different, we need to remove it
majors[majors['Major'] == 'Psychology, B.A.']['Text']

majors[50:90]

,Major,Text
58,"Mathematics, B.A.","Mathematics, B.A. Print-Friendly Page (opens a..."
59,"Media Studies, B.A.","Media Studies, B.A. Print-Friendly Page (opens..."
60,"Music, B.A.","Music, B.A. Print-Friendly Page (opens a new w..."
61,"Philosophy, B.A.","Philosophy, B.A. Print-Friendly Page (opens a ..."
62,"Physics, B.A.","Physics, B.A. Print-Friendly Page (opens a new..."
63,"Physics, B.S.","Physics, B.S. Print-Friendly Page (opens a new..."
64,"Psychology, B.A.","Psychology, B.A. Print-Friendly Page (opens a ..."
65,"Religious Studies, B.A.","Religious Studies, B.A. Print-Friendly Page (o..."
66,"Slavic Languages and Literatures, B.A.","Slavic Languages and Literatures, B.A. Print-F..."
67,"Sociology, B.A.","Sociology, B.A. Print-Friendly Page (opens a n..."


In [10]:
# remove extra text from the majors column where majors includes "interdisciplinary"
text_to_remove = 'Interdisciplinary - '
text_to_remove2 = 'Interdisciplinary Major - '
def remove_text_majors(text):
    if text.startswith(text_to_remove):
        text = text[len(text_to_remove):]
    if text.startswith(text_to_remove2):
        text = text[len(text_to_remove2):]
    return text.strip()

#apply the function to the majors column
majors['Major'] = majors['Major'].apply(remove_text_majors)

In [11]:
majors[20:40]

,Major,Text
28,"English, B.A.","English, B.A. Print-Friendly Page (opens a new..."
29,"Environmental Sciences, B.A.","Environmental Sciences, B.A. Print-Friendly Pa..."
30,"Environmental Sciences, B.S.","Environmental Sciences, B.S. Print-Friendly Pa..."
31,"Foreign Affairs, B.A.","Foreign Affairs, B.A. Print-Friendly Page (ope..."
32,"French, B.A.","French, B.A. Print-Friendly Page (opens a new ..."
33,"German, B.A.","German, B.A. Print-Friendly Page (opens a new ..."
34,"Government, B.A.","Government, B.A. Print-Friendly Page (opens a ..."
35,"History, B.A.","History, B.A. Print-Friendly Page (opens a new..."
36,American Studies,Interdisciplinary Major - American Studies Pri...
37,Archaeology,Interdisciplinary Major - Archaeology Print-Fr...


In [12]:
#read in schools data
schools = pd.read_csv("uva_schools_academic_rules.csv")
schools

,School,Text
0,School of Architecture,Info For Students Alumni Military Affiliated S...
1,College of Arts & Sciences,Info For Students Alumni Military Affiliated S...
2,McIntire School of Commerce,Info For Students Alumni Military Affiliated S...
3,School of Continuing and Professional Studies,Info For Students Alumni Military Affiliated S...
4,School of Data Science,Info For Students Alumni Military Affiliated S...
5,School of Education and Human Development,Info For Students Alumni Military Affiliated S...
6,School of Engineering and Applied Science,Info For Students Alumni Military Affiliated S...
7,Frank Batten School of Leadership and Public P...,Info For Students Alumni Military Affiliated S...
8,School of Nursing,Info For Students Alumni Military Affiliated S...


In [13]:
# remove the extra text from the text column
extra_begining_text = 'Info For Students Alumni Military Affiliated Students Faculty & Staff Search Search Submit Search Close search Info For Students Alumni Military Affiliated Students Faculty & Staff Calendars Academic Calendars Exam Schedules Student Records Diplomas Transcripts FERPA Programs Calendar Schools University Registrar Carruthers Hall, 1001 N. Emmet St. P.O. Box 400203 Charlottesville, VA 22904-4203 Staff Directory Contact Info Phone: (434) 924-4122 Fax: (434) 924-4156 Email: [email protected] Hours Of Operation M-F: 10am - noon and 1pm - 4pm Â© 2024 By the Rector and Visitors of the University of Virginia Legal Privacy Report a Barrier Share Your Feedback University of Virginia Mar 31, 2025 Undergraduate Record 2024-2025 Select a Catalog Undergraduate Record 2024-2025 Graduate Record 2024-2025 Global Search Catalog Search Choose Search Location Select an option Courses Programs Schools/Colleges & Departments Policies and Other Non-Academic Content Entire Catalog Search Keyword Field Whole Word/Phrase Advanced Search Catalog Navigation Catalog Home Academic Calendar Admission Schools Programs, Degrees & Course Info Student Resources ROTC University Regulations About UVA Archived Records HELP Undergraduate Record 2024-2025'

def remove_text_requirements(text):
    if text.startswith(extra_begining_text):
        text = text[len(extra_begining_text):]
    return text.strip()

#run the function on the text column
schools['Text'] = schools['Text'].apply(remove_text_requirements)

In [14]:
schools

,School,Text
0,School of Architecture,"School of Architecture: Academic Rules, Regula..."
1,College of Arts & Sciences,"College of Arts & Sciences: Academic Rules, Re..."
2,McIntire School of Commerce,"McIntire School of Commerce: Academic Rules, R..."
3,School of Continuing and Professional Studies,School of Continuing and Professional Studies:...
4,School of Data Science,School of Data Science: Academic Rules Print-F...
5,School of Education and Human Development,School of Education and Human Development: Aca...
6,School of Engineering and Applied Science,School of Engineering and Applied Science: Aca...
7,Frank Batten School of Leadership and Public P...,Frank Batten School of Leadership and Public P...
8,School of Nursing,"School of Nursing: Academic Rules, Regulations..."


# Add school metadata to majors and convert to dictionary

In [15]:
# copy of the majors data frame
majors_meta = majors
majors_meta['School'] = None
majors_meta = majors_meta.reset_index(drop=True)

#add school to each majors in major data frame
majors_meta['School'][0:3] = 'School of Architecture'
majors_meta['School'][3:62] = 'College of Arts and Sciences'
majors_meta['School'][62:63] = 'McIntire School of Commerce'
majors_meta['School'][63:65] = 'School of Continuing and Professional Studies'
majors_meta['School'][65:66] = 'School of Data Science'
majors_meta['School'][66:72] = 'School of Education and Human Development'
majors_meta['School'][72:83] = 'School of Engineering and Applied Science'
majors_meta['School'][83:84] = 'Frank Batten School of Leadership and Public Policy'
majors_meta['School'][84:85] = 'School of Nursing'

/tmp/ipykernel_19152/684139997.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  majors_meta['School'][0:3] = 'School of Architecture'
/tmp/ipykernel_19152/684139997.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pan

In [16]:
majors_meta.head(10)

,Major,Text,School
0,"Architectural History, B.Ar.H.","Architectural History, B.Ar.H. Print-Friendly ...",School of Architecture
1,"Architecture, B.S.","Architecture, B.S. Print-Friendly Page (opens ...",School of Architecture
2,"Urban and Environmental Planning, B.U.E.P.","Urban and Environmental Planning, B.U.E.P. Pri...",School of Architecture
3,"African-American and African Studies, B.A.","African American and African Studies, B.A. Pri...",College of Arts and Sciences
4,"Anthropology, B.A.","Anthropology, B.A. Print-Friendly Page (opens ...",College of Arts and Sciences
5,"Applied Statistics, B.A.","Applied Statistics, B.A. Print-Friendly Page (...",College of Arts and Sciences
6,"Area Studies, B.A.","Area Studies, B.A. Print-Friendly Page (opens ...",College of Arts and Sciences
7,Art History,History of Art Print-Friendly Page (opens a ne...,College of Arts and Sciences
8,Studio Art,Studio Art Print-Friendly Page (opens a new wi...,College of Arts and Sciences
9,"Astronomy, B.A.","Astronomy, B.A. Print-Friendly Page (opens a n...",College of Arts and Sciences


In [17]:
#convert majors to dictionary
majors_dict = majors_meta.to_dict(orient='records')
majors_dict

[{'Major': 'Architectural History, B.Ar.H.',
  'Text': 'Architectural History, B.Ar.H. Print-Friendly Page (opens a new window) Return to: School of Architecture: Degree Programs Universal Curriculum Requirements To be awarded a degree from the School of Architecture, students are required to complete universal curriculum requirements in addition to the program requirements provided below. The school universal curriculum requirements can be found on the school Degree Programs page . Program Requirements Undergraduate students entering the School of Architecture share a Common First Year in the School of Architecture . Students take core courses in Architectural History, Architecture, and Urban & Environmental Planning to provide a framework for the study of\xa0the built environment\xa0through observation, analysis, and design. Students must pass each core course with a grade of C- or higher. During the spring semester of the first year, students choose an intended major: Bachelor of Ar

In [18]:
#convert schools to dictionary
schools_dict = schools.to_dict(orient='records')
schools_dict

[{'School': 'School of Architecture',
  'Text': 'School of Architecture: Academic Rules, Regulations, and Information Print-Friendly Page (opens a new window) About Academic Rules Degree Programs Minor Programs Courses Click on a link to be taken to the entry below. Admission Requirements Academic Standing Appeals from Students Awards for Academic Excellence Curricular Requirements Course Rules Degree Information Direct Admissions Policy Grades Intra-University Transfers Leaving and Returning to the University Majors and Minors Public Professional Licensure Disclosure Student Intellectual Property Transfer Credit Admission Requirements All prospective undergraduate students apply through the University Office of Undergraduate Admissions; http://www.admission.virginia.edu/ For general information about the School of Architecture and undergraduate admissions: https://www.arch.virginia.edu/apply/undergraduate-admissions Academic Standing Academic Good Standing Students, in the School of A

# Chunking

In [19]:
# disable tokenizer parallelism so we stop getting problems
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [20]:
nltk.download("punkt")

# Load tokenizer (for OpenAI models)
tokenizer = tiktoken.get_encoding("cl100k_base")

# Function to chunk text while maintaining sentence structure
def chunk_text(text, max_tokens=500):
    sentences = nltk.tokenize.sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_tokens = len(tokenizer.encode(sentence))
        if current_length + sentence_tokens > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(sentence)
        current_length += sentence_tokens

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks



[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Store data in ChromaDB vectorized database

In [21]:
# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")  
collection = chroma_client.get_or_create_collection(name="Majors_Collection")

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

## Don't rerun adding info to database

In [ ]:
# Store chunked majors data
for course in majors_dict:
    chunks = chunk_text(course["Text"])
    for i, chunk in enumerate(chunks):
        embedding = model.encode(chunk).tolist()  # Convert to list for ChromaDB
        collection.add(
            ids=[f"{course['Major']}_chunk{i}"],  
            embeddings=[embedding],  
            metadatas=[{
                "Major": course["Major"],
                "chunk_index": i,
                "Text": chunk,
                "School": course['School']
            }]
        )

print("Chunked course descriptions stored in ChromaDB!")

In [ ]:
# Store chunked schools data
for course in schools_dict:
    chunks = chunk_text(course["Text"])
    for i, chunk in enumerate(chunks):
        embedding = model.encode(chunk).tolist()  # Convert to list for ChromaDB
        collection.add(
            ids=[f"{course['School']}_chunk{i}"],  
            embeddings=[embedding],  
            metadatas=[{
                "School": course["School"],
                "chunk_index": i,
                "Text": chunk
            }]
        )

print("Chunked schools stored in ChromaDB!")

# Retrieval

In [22]:
# function for document retrieval
#user can input the number of major documents and number of school documents they want to retrieve
def retrieve_major_and_school_info(query, k_m=10, k_s = 2):
    # Step 1: Retrieve major-related chunks
    query_embedding = model.encode(query).tolist()

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k_m
    )

    # Step 2: Extract major-specific results and school information
    major_info = []
    for i in range(len(results["ids"][0])):
        metadata = results["metadatas"][0][i]
        # If the metadata has 'major_name', this is a major document
        if metadata.get("Major"):
            major_info.append(metadata)

    # Step 3: Extract the school name from the major info
    schools_to_query = set([info["School"] for info in major_info])

    # Step 4: Retrieve relevant school descriptions based on the inferred school name
    school_info = []
    for school_name in schools_to_query:
        school_results = collection.query(
            query_embeddings=[query_embedding],
            where={"School": school_name},
            n_results=k_s
        )
        school_info.extend(school_results["metadatas"][0])

    return major_info, school_info

In [23]:
# Example search for "computer science"
query = "course schedule for computer science major"
major_info, school_info = retrieve_major_and_school_info(query, 10, 5)

In [24]:
major_info

[{'Major': 'Computer Science, B.S.',
  'School': 'School of Engineering and Applied Science',
  'Text': 'The 2000 level courses should be taken before the 3000 level courses and note that there are other prerequisites that govern the order that these courses should be taken. Example schedules can be found on the Computer Science Department webpage . CS 2100\xa0-\xa0Data Structures and Algorithms 1 Credits: 4 CS 2120\xa0-\xa0Discrete Mathematics and Theory 1 Credits: 3 CS 2130\xa0-\xa0Computer Systems and Organization 1 Credits: 4 CS 3100\xa0-\xa0Data Structures and Algorithms 2 Credits:          3 CS 3120\xa0-\xa0Discrete Mathematics and Theory 2 Credits: 3 CS 3130\xa0-\xa0Computer Systems and Organization 2 Credits: 4 CS 3140\xa0-\xa0Software Development Essentials Credits: 3 Upper-Level Required Courses BSCS majors must take one upper-level course in software engineering and one course to complete the SEAS senior thesis. CS 3240\xa0-\xa0Software Engineering Credits:          3 The se

In [25]:
school_info

[{'Major': 'Computer Science, B.S.',
  'School': 'School of Engineering and Applied Science',
  'Text': 'The 2000 level courses should be taken before the 3000 level courses and note that there are other prerequisites that govern the order that these courses should be taken. Example schedules can be found on the Computer Science Department webpage . CS 2100\xa0-\xa0Data Structures and Algorithms 1 Credits: 4 CS 2120\xa0-\xa0Discrete Mathematics and Theory 1 Credits: 3 CS 2130\xa0-\xa0Computer Systems and Organization 1 Credits: 4 CS 3100\xa0-\xa0Data Structures and Algorithms 2 Credits:          3 CS 3120\xa0-\xa0Discrete Mathematics and Theory 2 Credits: 3 CS 3130\xa0-\xa0Computer Systems and Organization 2 Credits: 4 CS 3140\xa0-\xa0Software Development Essentials Credits: 3 Upper-Level Required Courses BSCS majors must take one upper-level course in software engineering and one course to complete the SEAS senior thesis. CS 3240\xa0-\xa0Software Engineering Credits:          3 The se

In [26]:
#function to reformat major info into a long string
def format_major_info_string(major_info):
    formatted_texts = []
    for doc in major_info:
        school = doc.get('School', 'Unknown School') #get school or unknown school
        major = doc.get("Major", "Unknown Major")  # Get Major, or "Unknown Major" if missing
        text = doc.get("Text", "No Text Available")  # Get Text, or "No Text Available" if missing

        # Clean up newline characters and non-breaking spaces
        cleaned_text = text.replace("\n", " ").replace("\xa0", " ")

        formatted_texts.append(f"{school}: {major}: {cleaned_text}")

    context = "\n\n".join(formatted_texts)  # Join with double newlines for separation
    return context

In [27]:
#function to reformat school info into a long string
def format_school_info_string(school_info):
    formatted_texts = []
    for doc in school_info:
        school = doc.get('School', 'Unknown School') #get school or unknown school
        text = doc.get("Text", "No Text Available")  # Get Text, or "No Text Available" if missing

        # Clean up newline characters and non-breaking spaces
        cleaned_text = text.replace("\n", " ").replace("\xa0", " ")
        #cleaned_text = " ".join(text.split()) 

        formatted_texts.append(f"{school} \n {cleaned_text}")

    context = "\n\n".join(formatted_texts)  # Join with double newlines for separation
    return context

# Generation

In [28]:
#get token from .env file

# Load environment variables from .env file
load_dotenv()

# Retrieve the token
token = os.getenv("TOKEN")


In [29]:
# log into huggingface
login(token = token)

In [30]:
#set up torch
#if torch.backends.mps.is_available():
    #torch_device = 'mps'
if torch.cuda.is_available():
    torch_device = 'cuda'
else:
    torch_device = 'cpu'

In [31]:
#still too big

#load mistral model (don't run locally only run on AWS because of model size)
#model_checkpoint = "mistralai/Mistral-Small-3.1-24B-Instruct-2503"
#processor = AutoProcessor.from_pretrained(model_checkpoint)
#model = AutoModelForImageTextToText.from_pretrained( 
 #   model_checkpoint, 
  #  device_map = torch_device, 
   # torch_dtype = torch.float32
#)

In [ ]:
## to run it, becuase its not made to run on GPU??

import torch_neuronx

model_neuron = torch_neuronx.trace(model, example_inputs=torch.randint(0, 50257, (1, 1024)))
model_neuron.save("mistral_neuron.pt")

In [ ]:
# set system prompt
#set system prompt
SYSTEM_PROMPT = """You are a UVA undergraduate course scheduler. You use retrieved documents to create a schedule for one semester. 
Do not make up any courses, only use information from the documents. When you're not sure about some information, you say that you don't have the information and don't make up anything.
If the user's question is not clear, ambiguous, or does not provide enough context for you to accurately answer the question, you do not try to answer it right away and you rather ask the user to clarify their request.
A full time student should have between 12-15 credits per semester."""

In [ ]:
#function to generate response from LLM
def generate_response(query, k_m, k_s):
    '''retrieve relevant documents from majors and schools and generate response to user query using mistral 3 model'''

    #get documents
    major_info, school_info = retrieve_major_and_school_info(query, k_m, k_s)
    major = format_major_info_string(major_info)
    school = format_school_info_string(school_info)
    context = major + school

    #construct message
    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role':'user', 'content' : f"Context:\n{context}\n\nQuestion: {query}"}
    ]

    #process input for mistral
    text = processor.apply_chat_template(messages, tokenize = False, add_generation_prompt=True)
    inputs = processor(text=text, return_tensors='pt').to(torch_device, dtype = torch.float32)

    #generate response
    generate_id = model.generate(**inputs, max_new_tokens = 200, do_sample = False)
    decoded_output = processor.batch_decode(generate_id[:,inputs['input_ids'].shape[1]:], skip_special_tokens = True)[0]

    return decoded_output

In [ ]:
user_query = "I'm a computer science major at UVA and I need a schedule for the spring semester of my third year"
response = generate_response(user_query, 10, 5)